# Introduction

## Foreword


The following work was done in order to implement the recognized practices within the oil and gas industry task of calculating the maximal flow discharge throught the PSV (Pressure Safety Valve).
The script will include UI window with an option to export the calculated values to your desctop and deliver the excel list of the calculation done to the mentioned E-mail adress.

Important: Due to technical obstacles met during representing the UI window display on colaboratory environment, please consider running the code on jupyter lab environment.



First, we will import the nessesary packages:

In [21]:
from tkinter import *
import tkinter
import tkinter.messagebox
import pandas as pd
import os
from tkinter import ttk

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

The first step is about to create the main Tkinter window and define it's title




In [22]:
window = tkinter.Tk()
window.title("PSV Calculations")

''

Next step, we will create two frames, in order to be able to move to different windows which represent different calculation scenarios

In [23]:
notebook = ttk.Notebook(window)
notebook.pack(pady=10, expand=True)

frame1 = ttk.Frame(notebook)
frame2 = ttk.Frame(notebook)

frame1.pack(fill='both', expand=True)
frame2.pack(fill='both', expand=True)

# And add these frames to the notebook

notebook.add(frame1, text='Fire Case Discharge')
notebook.add(frame2, text='Thermal Expansion Dischare')

# Defining Variables



In order to be able to read the values from the corresponding input lines within the UI window, we will define the needed variables.

In [24]:
av_var=tkinter.StringVar()
SG_var=tkinter.StringVar()
fi_var=tkinter.StringVar()
c_var=tkinter.StringVar()
PSV_var=tkinter.StringVar()
E_mail_var=tkinter.StringVar()

Please note, the following lists will be defined in order to store the different calculation sessions and further will be stored in appropriate order within the excel file.

In [25]:
PSV_List=[]
Flow_List=[]
Units_List=[]

## The **calc** and **export** functions.

The **`calc`** function will include several lines of unit convertion alongside retrieving the variables from UI window. The unit convertion is with accordance to recognized engineering practices.

Further the function will perform the next actions:

*   Poping up a window delievering the calculated flow for this particular session

*   Appending the PSV number, Calculation result and Units to the coresponding list with each calculation session (each Blue botton activation)

*   Creating a **`data`** dictionary to store the values further imported to "Exported PSV List.xlsx" file on your desktop





In [26]:
def calc():


    #Retrieving the variables with tkinter.get method
    av=av_1.get()
    SG=SG_1.get()
    fi=fi_1.get()
    av_units=av_units_1.get()
    c=c_1.get()
    c_units=c_units_1.get()
    units=units_1.get()
    PSV=PSV_num_1.get()
    E_mail=E_mail_1.get()
    
    
   # Units convertion and calculating the q(FLOW)
   
    if av_units=='Kjoule/hr':
        av_converted=float(av)*0.947817
    if av_units=="Kcal/hr":
        av_converted=float(av)*3.96567
    if av_units=="Btu/hr":
        av_converted=float(av)



    if c_units=='Kjoule/KgK':
        c_converted=float(c)*0.239
    if c_units=="Joule/KgK":
        c_converted=float(c)*0.000239
    if c_units=="Btu/lb F":
        c_converted=float(c)
    if c_units=="Kcal/KgK":
        c_converted=float(c)
  


    if units=='Lit/hr':
        q=av_converted*float(fi)/500/float(SG)/c_converted*227.124
    if units=='M3/hr':
        q=av_converted*float(fi)/500/float(SG)/c_converted/264.17*60

    if units=='CFM':
        q=av_converted*float(fi)/500/float(SG)/c_converted/7.48
    if units=='USGPM':
        q=av_converted*float(fi)/500/float(SG)/c_converted
    

    
    #Pop up message box
    
    tkinter.messagebox.showinfo("The Flow is",  f'{"%.4f" % q} ''\nThe result will be saved in excel file on your desktop')

    PSV_List.append(PSV)
    Flow_List.append(q)
    Units_List.append(units)  
    
    #Create a dictionary

    data = {"PSV Number": PSV_List,
            "Calculated Flow": Flow_List,
            "Flow Units":Units_List
          }
    #Create a dataframe

    df = pd.DataFrame(data, columns = ['PSV Number', 'Calculated Flow','Flow Units'])

    #Retrieving the users Desktop path
    #If your OS is other than windows, consider to change the path finder correspondingly

    a=os.path.join(os.environ['HOMEPATH'], 'Desktop')
    # print(str(a)+'\Exported PSV List.xlsx')
    
    #Creating the excel file with the calculation results
    #It will be updated each calculation you perform
    

    df.to_excel (str(a)+'\Exported PSV List.xlsx', index = False, header=True)
    
    #print(E_mail)
    return q

The **`export`** function will export the created excel file to the mentioned e-mail adress.
Hence, the consequense of actions should be done throughout the calculation and further exporting with the red button in the UI window.

In [27]:
def export():
    
    # Retrieving the email adress from the entry line window:
    E_mail=E_mail_1.get() 

    # E-mail settings(Please note, the sender email and the password lines should be updated with accordance with your user data):
    subject = "Exported Flow Calculation"
    body = "Please find attached the excel file with the overall calculation session results."
    sender_email = "********@gmail.com"
    receiver_email =E_mail
    password = "***********"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email 

    # Add body to email and define the file path
    message.attach(MIMEText(body, "plain"))
    a=os.path.join(os.environ['HOMEPATH'], 'Desktop')
    filename = str(a)+'\Exported PSV List.xlsx'
#   'Exported PSV List.xlsx'  # Should be in same directory as script

    
    # Open XLSX file:
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    
    return

# UI Design with tkinter pacakge

There are multiple widgets such as:


*   Entry lines
*   Drop down menu
*   Buttons
*   Check boxes





## Creating widgets

In [ ]:
# You will create several labels named 'SG'(Specific Gravity) and 
# 'av'(Heat transfer rate) ect.. (+ Entry windows for them)
tkinter.Label(frame1, text = "SG").grid(row = 1) # placed on position 10 (row - 1 and column - 0)

# 'Entry' class is used to display the input-field for 'username' text label
SG_1=tkinter.Entry(frame1,textvariable=SG_var)# Text variable(refer to Define Variables section)
SG_1.grid(row = 1, column = 1) # first input-field is placed next to the label on position 11 (row - 1 and column - 1)




tkinter.Label(frame1, text = "av-Heat transfer rate to the vessel").grid(row = 2)
av_1=tkinter.Entry(frame1,textvariable=av_var) 
av_1.grid(row = 2, column = 1)



options = tkinter.StringVar(frame1)
options.set("Kjoule/hr") # default value

av_units_label = tkinter.Label(frame1,  text='', width=15 )  
av_units_label.grid(row=3,column=1) 

#Defining class for Dropdown menu variables retriever
class DropDown(tkinter.OptionMenu):
    """
    Classic drop down entry

    Example use:
        # create the dropdown and grid
        av_units = DropDown(frame1,["Kjoule/hr","Kcal/hr", "Btu/hr"])
        av_units.grid(row=5,column=2)

        # define a callback function that retrieves the currently selected option
        def callback():
            print(av_units.get())

        # add the callback function to the dropdown
        av_units.add_callback(callback)
    """
    def __init__(self, parent, options: list, initial_value: str=None):
        """
        Constructor for drop down entry

        :param parent: the tk parent frame
        :param options: a list containing the drop down options
        :param initial_value: the initial value of the dropdown
        """
        self.var = tkinter.StringVar(parent)
        self.var.set(initial_value if initial_value else options[0])
        self.option_menu = tkinter.OptionMenu.__init__(self, parent, self.var, *options)
        self.callback = None

    def add_callback(self, callback: callable):
        """
        Add a callback on change

        :param callback: callable function
        :return: 
        """
        def internal_callback(*args):
            callback()

        self.var.trace("w", internal_callback)

    def get(self):
        """
        Retrieve the value of the dropdown

        :return: 
        """
        return self.var.get()

    def set(self, value: str):
        """
        Set the value of the dropdown

        :param value: a string representing the
        :return: 
        """
        self.var.set(value)


        
        
        
av_units_1 = DropDown(frame1,["Kjoule/hr","Kcal/hr", "Btu/hr"])
av_units_1.grid(row=2,column=2)      
        
        
#fi - 1/F Cubic Expansion coefficient of the fluid

tkinter.Label(frame1, text = "Cubic Expansion coefficient of the fluid").grid(row = 3) 
fi_1=tkinter.Entry(frame1,textvariable=fi_var)
fi_1.grid(row = 3, column = 1) #second input-field is placed on position 11 (row - 1 and column - 1)
tkinter.Label(frame1, text = "[1/F]").grid(row = 3,column=2)


#c=float(input('Insert Specific heat of the fluid\n'))
#c_units=input('Insert c units: [Kjoule/KgK] or [Btu/lb F] or [Kcal/KgK] or [Joule/KgK]\n')

tkinter.Label(frame1, text = "Specific heat of the fluid").grid(row = 7) 
c_1=tkinter.Entry(frame1,textvariable=c_var)
c_1.grid(row = 7, column = 1)


options = tkinter.StringVar(frame1)
options.set("Kjoule/KgK") # default value

# c_units_1 =tkinter.OptionMenu(window, options, "Kjoule/KgK","Btu/lb F", "Kcal/KgK","Joule/KgK")
# c_units_1.grid(row=8,column=2) 

c_units_1 = DropDown(frame1,["Kjoule/KgK","Btu/lb F", "Kcal/KgK",'Joule/KgK'])
c_units_1.grid(row=7,column=2)


#units=input('Insert flow calc units: [Lit/hr] or [M3/hr] or [CFM] or [USGPM]\n')

options = tkinter.StringVar(frame1)
options.set("Lit/hr") # default value

l4 = tkinter.Label(frame1,  text='Flow Units to be Calculated', width=20 )  
l4.grid(row=9,column=1) 

# units =tkinter.OptionMenu(window, options, "Lit/hr","CFM", "M3/hr","USGPM")
# units.grid(row=9,column=2) 

units_1 = DropDown(frame1,["Lit/hr","CFM", "M3/hr",'USGPM'])
units_1.grid(row=9,column=2)


# Calculation Button(define the function name in command attribute)

button = tkinter.Button(frame1, text='Calculate the Flow',bg='blue',fg='white',width=17, command=calc)
button.grid(row=10, column=0, pady=(0,7))

# PSV number for listing the results for each particular unit:

PSV_num_2 = tkinter.Label(frame1,  text='Enter the PSV Tag Number', width=20 )  
PSV_num_2.grid(row=10,column=1) 

PSV_num_1=tkinter.Entry(frame1,textvariable=PSV_var)
PSV_num_1.grid(row = 10, column = 2) 


# Export Button(define the function name in command attribute)

button2 = tkinter.Button(frame1, text='Export via E-mail',bg='red',fg='white',width=17, command=export)
button2.grid(row=11, column=0, pady=(0,7))

E_mail = tkinter.Label(frame1,  text='Enter your E-mail', width=20 )  
E_mail.grid(row=11,column=1) 

E_mail_1=tkinter.Entry(frame1,textvariable=E_mail_var)
E_mail_1.grid(row = 11, column = 2) #second input-field is placed on position 11 (row - 1 and column - 1)










# # 'Checkbutton' class is for creating a checkbutton which will take a 
# 'columnspan' of width two (covers two columns)
# Please note that the checkbutton is placed on frame2(sheet)
tkinter.Checkbutton(frame2, text = "Keep Me Logged In").grid(columnspan = 5)                 



window.mainloop()  # Keeps the main window open
window.mainloop